            TODO 
-> save models to further usage
-> player

In [2]:
import pysrt
import joblib


model = joblib.load('C:\dev\whisper_models\whisper_small.pkl')

In [29]:
import os

In [28]:
import whisper
import pysrt

model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [18:36<00:00, 1.37MiB/s]


In [30]:
result = model.transcribe("C:\Movies\My Hero Academia S01 (2016) MULTi 1080p 10bits BluRay x265 AAC v2 -Punisher694\My Hero Academia S01E02 MULTi 1080p 10bits BluRay x265 AAC -Punisher694.mkv")
print(result["text"])

In [4]:
def convert_time(seconds):

    hours = int(seconds // 3600)
    remaining_seconds = seconds % 3600
    
    minutes = int(remaining_seconds // 60)
    remaining_seconds %= 60
    
    seconds = int(remaining_seconds)
    miliseconds = int((remaining_seconds - seconds) * 1000)
    
    return (hours, minutes, seconds, miliseconds)


In [8]:
def result_to_srt(result, output_file):
    subs = pysrt.SubRipFile()
    
    for segment in result['segments']:
        start_time = convert_time(segment['start']) 
        end_time = convert_time(segment['end'])
        
        sub_item = pysrt.SubRipItem(start=start_time, 
                                    end=end_time, 
                                    text=segment['text'])
        subs.append(sub_item)
    subs.save(output_file, encoding='utf-8')
    print(f'Subtitle file {output_file} is ready.')   

In [10]:
result_to_srt(result, 'output.srt')

In [ ]:
def subtitles_for_folder(video_folder, sub_folder):
    files = os.listdir(video_folder)
    file_count = len(files)
    done = 0
    for file in files:
        video_path = os.path.join(video_folder, file)
        result = model.transcribe(video_path)
        
        sub_file = os.path.splittext(video_path)[0] + ".srt"
        sub_path = os.path.join(sub_folder, sub_file)
        result_to_srt(result, sub_path)
        done += 1
        print(f"{done}/{file_count}")
        